# 0. 환경설정

## 0-1. 패키지 설치

In [ ]:
!pip install langchain
!pip install langchain-neo4j
!pip install langchain-community
!pip install neo4j>=5.0.0
d
!pip install langchain-openai
!pip install langchain-google-genai
!pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.5.0
    Uninstalling tenacity-8.5.0:
      Successfully uninstalled tenacity-8.5.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.7.0 requires fsspec==2025.7.0, but you have fsspec 2024.12.0 which is incompatible.
google-genai 1.26.0 requires tenacity<9.0.0,>=8.2.3, but you have tenacity 9.1.2 which

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.3/754.3 kB 7.5 MB/s eta 0:00:00


## 0-2. NEO4j DB 연결

In [ ]:
# Google Colab의 userdata 사용
from langchain_neo4j import Neo4jGraph
from google.colab import userdata

# Neo4j 연결 (Colab Secrets 사용)
try:
    graph = Neo4jGraph(
        url=userdata.get('NEO4J_URI'),
        username=userdata.get('NEO4J_USERNAME'),
        password=userdata.get('NEO4J_PASSWORD')
    )
    print("✅ Neo4j 연결 성공!")
except Exception as e:
    print(f"❌ Neo4j 연결 실패: {e}")

✅ Neo4j 연결 성공!


In [ ]:
# 테스트 쿼리 실행
cypher_query = """
CREATE (n:Test {name: "Hello AuraDB"})
RETURN n
"""

graph.query(cypher_query)

[{'n': {'name': 'Hello AuraDB'}}]

## 0-3. DB 초기화

In [ ]:
def reset_database(graph):
    """
    데이터베이스 초기화하기
    """
    # 모든 노드와 관계 삭제
    graph.query("MATCH (n) DETACH DELETE n")

    # 모든 제약조건 삭제
    constraints = graph.query("SHOW CONSTRAINTS")
    for constraint in constraints:
        constraint_name = constraint.get("name")
        if constraint_name:
            graph.query(f"DROP CONSTRAINT {constraint_name}")

    # 모든 인덱스 삭제
    indexes = graph.query("SHOW INDEXES")
    for index in indexes:
        index_name = index.get("name")
        index_type = index.get("type")
        if index_name and index_type != "CONSTRAINT":
            graph.query(f"DROP INDEX {index_name}")

    print("데이터베이스가 초기화되었습니다.")

# 데이터베이스 초기화
reset_database(graph)

데이터베이스가 초기화되었습니다.


# 1. 지식 그래프 구축

## 1-1. 제약 조건 생성
- Page 노드의 url 속성에 고유성 제약조건 설

In [ ]:
# 새로운 제약조건
new_constraints = [
    # ROOT 도메인 고유성
    "CREATE CONSTRAINT root_domain_unique IF NOT EXISTS FOR (r:ROOT) REQUIRE r.domain IS UNIQUE",

    # PAGE ID 고유성
    "CREATE CONSTRAINT page_id_unique IF NOT EXISTS FOR (p:PAGE) REQUIRE p.pageId IS UNIQUE"
]


for constraint in new_constraints:
    try:
        graph.query(constraint)
        print(f"✅ 제약조건 생성: {constraint.split('CREATE CONSTRAINT')[1].split('IF')[0].strip()}")
    except Exception as e:
        print(f"⚠️ {e}")

✅ 제약조건 생성: root_domain_unique
✅ 제약조건 생성: page_id_unique


## 1-2. 인덱스 생성(검색 성능 최적화) - TODO

In [ ]:
# 새로운 인덱스
new_indexes = [
    # PAGE URL 인덱스
    "CREATE INDEX page_url_index IF NOT EXISTS FOR (p:PAGE) ON (p.url)",

    # 전문 검색 인덱스 (자연어 매칭)
    "CREATE FULLTEXT INDEX page_search IF NOT EXISTS FOR (p:PAGE) ON EACH [p.textLabels]"
]

for index in new_indexes:
    try:
        graph.query(index)
        print(f"✅ 인덱스 생성: {index.split('CREATE')[1].split('INDEX')[1].split('IF')[0].strip()}")
    except Exception as e:
        print(f"⚠️ {e}")

✅ 인덱스 생성: page_url_index
✅ 인덱스 생성: page_search


## 1-3. JSONLoader 설정

In [ ]:
from langchain_community.document_loaders import JSONLoader
import json

# 예시 JSON - "유튜브에서 좋아요 한 음악 재생목록 열기" 경로
example_json = {
    "sessionId": "session_20250123_001",
    "startCommand": "유튜브에서 좋아요 한 음악 재생목록 열기",
    "completePath": [
        {
            "order": 0,
            "url": "https://youtube.com",
            "clickedElement": None  # 시작점이므로 클릭 요소 없음
        },
        {
            "order": 1,
            "url": "https://youtube.com",
            "locationData": {
                "primarySelector": "button[aria-label='라이브러리']",
                "fallbackSelectors": [
                    "ytd-mini-guide-entry-renderer:nth-child(4) button",
                    "button:has-text('라이브러리')"
                ],
                "anchorPoint": "#guide-renderer",
                "relativePathFromAnchor": "button[aria-label='라이브러리']",
                "elementSnapshot": {
                    "tagName": "button",
                    "attributes": {
                        "id": "endpoint",
                        "aria-label": "라이브러리",
                        "class": "style-scope ytd-mini-guide-entry-renderer"
                    }
                }
            },
            "semanticData": {
                "textLabels": [
                    "라이브러리",
                    "Library"
                ],
                "contextText": {
                    "immediate": "YouTube 가이드",
                    "section": "메인 네비게이션",
                    "neighbor": ["홈", "Shorts", "구독"]
                },
                "pageInfo": {
                    "title": "YouTube",
                    "url": "https://youtube.com"
                },
                "actionType": "click"
            }
        },
        {
            "order": 2,
            "url": "https://youtube.com/feed/library",
            "locationData": {
                "primarySelector": "tp-yt-paper-tab[aria-label='재생목록']",
                "fallbackSelectors": [
                    "paper-tab:nth-child(3)",
                    "tp-yt-paper-tab:has-text('재생목록')"
                ],
                "anchorPoint": "#tabs-container",
                "relativePathFromAnchor": "tp-yt-paper-tab:nth-child(3)",
                "elementSnapshot": {
                    "tagName": "tp-yt-paper-tab",
                    "attributes": {
                        "aria-label": "재생목록",
                        "role": "tab"
                    }
                }
            },
            "semanticData": {
                "textLabels": [
                    "재생목록",
                    "Playlists"
                ],
                "contextText": {
                    "immediate": "라이브러리 탭",
                    "section": "YouTube 라이브러리",
                    "neighbor": ["기록", "동영상", "나중에 볼 동영상", "좋아요 표시한 동영상"]
                },
                "pageInfo": {
                    "title": "라이브러리 - YouTube",
                    "url": "https://youtube.com/feed/library"
                },
                "actionType": "click"
            }
        },
        {
            "order": 3,
            "url": "https://youtube.com/feed/library/playlists",
            "locationData": {
                "primarySelector": "a[title='좋아요 표시한 음악']",
                "fallbackSelectors": [
                    "ytd-playlist-thumbnail a[href*='LM']",
                    "#content a:has-text('좋아요 표시한 음악')"
                ],
                "anchorPoint": "#contents",
                "relativePathFromAnchor": "ytd-grid-renderer a[title='좋아요 표시한 음악']",
                "elementSnapshot": {
                    "tagName": "a",
                    "attributes": {
                        "href": "/playlist?list=LM",
                        "title": "좋아요 표시한 음악"
                    }
                }
            },
            "semanticData": {
                "textLabels": [
                    "좋아요 표시한 음악",
                    "Liked Music",
                    "자동 재생목록"
                ],
                "contextText": {
                    "immediate": "재생목록",
                    "section": "내 재생목록",
                    "neighbor": ["나중에 볼 동영상", "새 재생목록"]
                },
                "pageInfo": {
                    "title": "재생목록 - YouTube",
                    "url": "https://youtube.com/feed/library/playlists"
                },
                "actionType": "click"
            }
        }
    ]
}

## 1-4. 메타데이터
- i. json.dumps() vs json.dump()
  - json.dumps() : python 객체를 json 형식의 문자열로 변환
  - dumps의 s는 string을 의미
  - 메모리에 문자열로 저장
- json.dump() : python 객체를 json 형식으로 변환하여 파일에 직접 저장
  - 파일 객체가 필요

- ii. sort_keys=True
  - 딕셔너리 키를 정렬
  - 동일한 데이터는 항상 동일한 문자열로 변환되도록 보장
  - 해시값 생성 시 일관성 유지에 중요

- iii. textLabels : 텍스트 레이블들을 공백으로 연결

- iv. immediate : 즉각적인 컨텍스트 텍스트

-  v. section : 섹션 컨텍스트 텍스트

- vi. .get() :  키가 없는 경우

In [ ]:
from datetime import datetime
import hashlib
import json

def add_metadata_to_path(path_json):

  path_string = json.dumps(path_json['completePath'], sort_keys=True)

  path_id = hashlib.md5(
      f"{path_json['startCommand']}_{path_string}".encode()
  ).hexdigest()

  # 메타데이터 추가
  path_json['metadata'] = {
      'pathId': path_id,
      'createAt': datetime.now().isoformat(),
      'lastUsed': datetime.now().isoformat(),
      'usageCount': 1,
      'successRate': 1.0, # 초기값
      'avgExecutionTime': None, # 아직 실행 안됨
      'createBy': 'user_001',
      'status': 'active' # active, deprecated, broken
  }

  # 각 스탭에도 임베딩을 위한 텍스트 준비
  for step in path_json['completePath']:
    if step['order'] > 0:

      semantic_text = ' '.join([
          ' '.join(step['semanticData'].get('textLabels', [])),
          step['semanticData'].get('contextText', {}).get('immediate', ''),
          step['semanticData'].get('contextText', {}).get('section', ''),
      ])
      neighbor_text = ' '.join(step['semanticData'].get('contextText', {}).get('neighbor', []))
      if neighbor_text:
          semantic_text = f"{semantic_text} {neighbor_text}"

      page_title = step['semanticData'].get('pageInfo', {}).get('title', '')
      if page_title:
          semantic_text = f"{semantic_text} {page_title}"

      semantic_text = ' '.join(semantic_text.split())

      step['embeddingText'] = semantic_text
  return path_json

In [ ]:
enriched_json = add_metadata_to_path(example_json)

print(json.dumps(enriched_json['metadata'], indent=2))

{
  "pathId": "ebfbfd5b2329a591187ea90842266945",
  "createAt": "2025-07-25T05:10:27.176840",
  "lastUsed": "2025-07-25T05:10:27.176853",
  "usageCount": 1,
  "successRate": 1.0,
  "avgExecutionTime": null,
  "createBy": "user_001",
  "status": "active"
}


## 1-5. 유틸리티 함수

In [ ]:
import hashlib
from urllib.parse import urlparse

def extract_domain(url):
    """URL에서 도메인 추출"""
    parsed = urlparse(url)
    return parsed.netloc.replace('www.', '')

def create_page_id(url, element_data):
    """PAGE 노드의 고유 ID 생성"""
    # URL + 요소 정보로 고유 ID 생성
    element_key = (
        element_data.get('primarySelector', '') +
        '_' +
        str(element_data.get('textLabels', []))
    )
    return hashlib.md5(f"{url}_{element_key}".encode()).hexdigest()

## 1-6. 임베딩 모델 초기화

In [ ]:
from openai import OpenAI
import numpy as np

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def generate_embedding(text):
    try:
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=text
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"임베딩 생성 실패: {e}")
        return None

## 1-7. 시멘틱 데이터를 임베딩 텍스트로 변환

In [ ]:
def create_embedding_text(step_data):
    """PAGE 임베딩용 텍스트 생성"""
    texts = []

    # textLabels
    texts.extend(step_data['semanticData']['textLabels'])

    # contextText
    context = step_data['semanticData']['contextText']
    texts.append(context.get('immediate', ''))
    texts.append(context.get('section', ''))
    texts.extend(context.get('neighbor', []))

    # pageInfo
    texts.append(step_data['semanticData']['pageInfo']['title'])

    return ' '.join(filter(None, texts))

## 1-8. 노드 생성

In [ ]:
def is_base_url(url):
    """URL이 base domain인지 확인"""
    parsed = urlparse(url)
    # 경로가 비어있거나 '/'만 있으면 base URL
    return parsed.path in ['', '/'] and not parsed.query and not parsed.fragment

In [ ]:
def save_desire_path_fixed(graph, path_json):
    """클릭 요소 우선 확인하는 수정된 버전"""

    clicks = [step for step in path_json['completePath'] if step['order'] > 0]

    if not clicks:
        print("❌ 클릭 데이터가 없습니다")
        return

    previous_node_id = None
    previous_node_type = None
    previous_domain = None

    for i, step in enumerate(clicks):
        current_url = step['url']
        current_domain = extract_domain(current_url)
        domain_changed = previous_domain and previous_domain != current_domain

        # 핵심 수정: 클릭 요소가 있는지 먼저 확인!
        has_click_element = 'locationData' in step and 'semanticData' in step

        if has_click_element:
            # 클릭 요소가 있으면 무조건 PAGE로 생성
            page_id = create_page_id(current_url, step['locationData'])
            embedding_text = create_embedding_text(step)
            embedding = generate_embedding(embedding_text)

            create_page_query = """
            MERGE (p:PAGE {pageId: $pageId})
            SET p.url = $url,
                p.domain = $domain,
                p.primarySelector = $primarySelector,
                p.fallbackSelectors = $fallbackSelectors,
                p.textLabels = $textLabels,
                p.contextText = $contextText,
                p.actionType = $actionType,
                p.embedding = $embedding,
                p.lastUpdated = datetime()
            RETURN p
            """

            page_params = {
                'pageId': page_id,
                'url': current_url,
                'domain': current_domain,
                'primarySelector': step['locationData']['primarySelector'],
                'fallbackSelectors': step['locationData']['fallbackSelectors'],
                'textLabels': step['semanticData']['textLabels'],
                'contextText': json.dumps(step['semanticData']['contextText']),
                'actionType': step['semanticData']['actionType'],
                'embedding': embedding
            }

            graph.query(create_page_query, page_params)
            print(f"✅ PAGE: {step['semanticData']['textLabels'][0]} ({current_domain})")

            # 연결 로직
            if i == 0 or domain_changed:
                # 첫 PAGE이거나 도메인 변경 시 ROOT 연결
                create_root_if_needed = """
                MERGE (r:ROOT {domain: $domain})
                SET r.baseURL = $baseURL,
                    r.lastVisited = datetime()
                RETURN r
                """
                graph.query(create_root_if_needed, {
                    'domain': current_domain,
                    'baseURL': f"https://{current_domain}"
                })

                connect_to_root = """
                MATCH (r:ROOT {domain: $domain})
                MATCH (p:PAGE {pageId: $pageId})
                MERGE (r)-[rel:HAS_PAGE]->(p)
                SET rel.weight = coalesce(rel.weight, 0) + 1
                """
                graph.query(connect_to_root, {
                    'domain': current_domain,
                    'pageId': page_id
                })

                # 도메인 간 연결
                if previous_node_type == 'PAGE' and domain_changed:
                    cross_connect = """
                    MATCH (p1:PAGE {pageId: $prevId})
                    MATCH (p2:PAGE {pageId: $currId})
                    MERGE (p1)-[rel:NAVIGATES_TO_CROSS_DOMAIN]->(p2)
                    SET rel.weight = coalesce(rel.weight, 0) + 1
                    """
                    graph.query(cross_connect, {
                        'prevId': previous_node_id,
                        'currId': page_id
                    })

            elif previous_node_type == 'PAGE':
                # PAGE → PAGE 연결
                connect_pages = """
                MATCH (p1:PAGE {pageId: $prevId})
                MATCH (p2:PAGE {pageId: $currId})
                MERGE (p1)-[rel:NAVIGATES_TO]->(p2)
                SET rel.weight = coalesce(rel.weight, 0) + 1
                """
                graph.query(connect_pages, {
                    'prevId': previous_node_id,
                    'currId': page_id
                })

            previous_node_type = 'PAGE'
            previous_node_id = page_id

        elif is_base_url(current_url):
            # 클릭 요소 없고 base URL인 경우만 ROOT
            print(f"✅ ROOT: {current_domain} (클릭 요소 없음)")
            # ROOT 처리 로직...

        previous_domain = current_domain

    print("\n✅ 경로 저장 완료!")



In [ ]:
# 테스트 케이스 1: YouTube에서 음악 검색 후 필터 적용
test_case_1 = {
    "sessionId": "session_20250123_test1",
    "startCommand": "유튜브에서 최신 음악 영상 찾기",
    "completePath": [
        {
            "order": 0,
            "url": "https://youtube.com",
            "clickedElement": None
        },
        {
            "order": 1,
            "url": "https://youtube.com",
            "locationData": {
                "primarySelector": "input#search",
                "fallbackSelectors": ["ytd-searchbox input", "input[name='search_query']"],
                "anchorPoint": "#masthead",
                "relativePathFromAnchor": "input#search",
                "elementSnapshot": {"tagName": "input", "attributes": {"id": "search"}}
            },
            "semanticData": {
                "textLabels": ["검색", "Search"],
                "contextText": {
                    "immediate": "검색창",
                    "section": "헤더",
                    "neighbor": ["YouTube", "마이크"]
                },
                "pageInfo": {"title": "YouTube", "url": "https://youtube.com"},
                "actionType": "click"
            }
        },
        {
            "order": 2,
            "url": "https://youtube.com/results?search_query=음악",
            "locationData": {
                "primarySelector": "button[aria-label='검색 필터']",
                "fallbackSelectors": ["#filter-button", "ytd-search-filter-renderer button"],
                "anchorPoint": "#container",
                "relativePathFromAnchor": "button[aria-label='검색 필터']",
                "elementSnapshot": {"tagName": "button"}
            },
            "semanticData": {
                "textLabels": ["필터", "검색 필터"],
                "contextText": {
                    "immediate": "검색 도구",
                    "section": "검색 결과",
                    "neighbor": ["정렬 기준", "모든 동영상"]
                },
                "pageInfo": {"title": "음악 - YouTube", "url": "https://youtube.com/results?search_query=음악"},
                "actionType": "click"
            }
        },
        {
            "order": 3,
            "url": "https://youtube.com/results?search_query=음악&sp=CAI%253D",
            "locationData": {
                "primarySelector": "a[aria-label='오늘']",
                "fallbackSelectors": ["yt-chip-cloud-chip-renderer:contains('오늘')"],
                "anchorPoint": "#chips",
                "relativePathFromAnchor": "a[aria-label='오늘']",
                "elementSnapshot": {"tagName": "a"}
            },
            "semanticData": {
                "textLabels": ["오늘", "Today"],
                "contextText": {
                    "immediate": "업로드 날짜",
                    "section": "필터",
                    "neighbor": ["이번 주", "이번 달", "올해"]
                },
                "pageInfo": {"title": "음악 - YouTube", "url": "https://youtube.com/results?search_query=음악&sp=CAI%253D"},
                "actionType": "click"
            }
        },
        {
            "order": 4,
            "url": "https://youtube.com/results?search_query=음악&sp=EgIIAg%253D%253D",
            "locationData": {
                "primarySelector": "ytd-video-renderer:first-child a#video-title",
                "fallbackSelectors": ["a#video-title:first", "h3.title-and-badge a"],
                "anchorPoint": "#contents",
                "relativePathFromAnchor": "ytd-video-renderer:first-child a",
                "elementSnapshot": {"tagName": "a", "attributes": {"id": "video-title"}}
            },
            "semanticData": {
                "textLabels": ["첫 번째 동영상", "최신 음악"],
                "contextText": {
                    "immediate": "동영상 제목",
                    "section": "검색 결과",
                    "neighbor": ["조회수", "업로드 날짜"]
                },
                "pageInfo": {"title": "음악 (오늘) - YouTube", "url": "https://youtube.com/results?search_query=음악&sp=EgIIAg%253D%253D"},
                "actionType": "click"
            }
        }
    ]
}

# 테스트 케이스 2: YouTube에서 음악 검색 후 바로 선택 (일부 경로 공유)
test_case_2 = {
    "sessionId": "session_20250123_test2",
    "startCommand": "유튜브에서 인기 음악 영상 보기",
    "completePath": [
        {
            "order": 0,
            "url": "https://youtube.com",
            "clickedElement": None
        },
        {
            "order": 1,
            "url": "https://youtube.com",
            "locationData": {
                "primarySelector": "input#search",
                "fallbackSelectors": ["ytd-searchbox input", "input[name='search_query']"],
                "anchorPoint": "#masthead",
                "relativePathFromAnchor": "input#search",
                "elementSnapshot": {"tagName": "input", "attributes": {"id": "search"}}
            },
            "semanticData": {
                "textLabels": ["검색", "Search"],
                "contextText": {
                    "immediate": "검색창",
                    "section": "헤더",
                    "neighbor": ["YouTube", "마이크"]
                },
                "pageInfo": {"title": "YouTube", "url": "https://youtube.com"},
                "actionType": "click"
            }
        },
        {
            "order": 2,
            "url": "https://youtube.com/results?search_query=음악",
            "locationData": {
                "primarySelector": "ytd-video-renderer:nth-child(3) a#video-title",
                "fallbackSelectors": ["#contents ytd-video-renderer:nth-child(3) a"],
                "anchorPoint": "#contents",
                "relativePathFromAnchor": "ytd-video-renderer:nth-child(3) a",
                "elementSnapshot": {"tagName": "a", "attributes": {"id": "video-title"}}
            },
            "semanticData": {
                "textLabels": ["세 번째 동영상", "인기 음악"],
                "contextText": {
                    "immediate": "동영상 제목",
                    "section": "검색 결과",
                    "neighbor": ["조회수 1M", "3일 전"]
                },
                "pageInfo": {"title": "음악 - YouTube", "url": "https://youtube.com/results?search_query=음악"},
                "actionType": "click"
            }
        },
        {
            "order": 3,
            "url": "https://youtube.com/watch?v=abc123",
            "locationData": {
                "primarySelector": "button[aria-label='좋아요']",
                "fallbackSelectors": ["#top-level-buttons button:first-child", "like-button-view-model button"],
                "anchorPoint": "#actions",
                "relativePathFromAnchor": "button[aria-label='좋아요']",
                "elementSnapshot": {"tagName": "button"}
            },
            "semanticData": {
                "textLabels": ["좋아요", "Like"],
                "contextText": {
                    "immediate": "동영상 액션",
                    "section": "플레이어",
                    "neighbor": ["싫어요", "공유", "저장"]
                },
                "pageInfo": {"title": "인기 음악 - YouTube", "url": "https://youtube.com/watch?v=abc123"},
                "actionType": "click"
            }
        }
    ]
}

In [ ]:
def run_overlap_test(graph, test_cases):
    """경로 겹침 테스트 실행"""

    print("🧪 경로 겹침 테스트 시작\n")

    # 각 테스트 케이스 실행
    for i, test_case in enumerate(test_cases, 1):
        print(f"📌 테스트 케이스 {i}: {test_case['startCommand']}")
        test_with_metadata = add_metadata_to_path(test_case.copy())
        save_desire_path_with_domain_handling(graph, test_with_metadata)
        print()

    # 결과 분석
    print("\n📊 테스트 결과 분석:")

    # 1. 공유되는 노드 확인
    shared_nodes = graph.query("""
    MATCH (p:PAGE)<-[:HAS_PAGE|NAVIGATES_TO*]-(r:ROOT)
    WITH p, count(DISTINCT r) as root_count
    WHERE root_count > 0
    MATCH (p)<-[rel:HAS_PAGE|NAVIGATES_TO]-()
    WITH p, count(rel) as incoming_count
    WHERE incoming_count > 1
    RETURN p.textLabels[0] as page, p.url as url, incoming_count
    """)

    print("\n🔄 공유되는 PAGE 노드:")
    for node in shared_nodes:
        print(f"  - {node['page']}: {node['incoming_count']}개 경로에서 사용")

    # 2. 경로 가중치 확인
    weights = graph.query("""
    MATCH ()-[rel:HAS_PAGE|NAVIGATES_TO]->()
    WHERE rel.weight > 1
    MATCH (from)-[rel]->(to)
    RETURN
        labels(from)[0] as from_type,
        CASE
            WHEN from:ROOT THEN from.domain
            ELSE from.textLabels[0]
        END as from_name,
        labels(to)[0] as to_type,
        CASE
            WHEN to:ROOT THEN to.domain
            ELSE to.textLabels[0]
        END as to_name,
        rel.weight as weight
    ORDER BY rel.weight DESC
    """)

    print("\n⚖️ 증가된 가중치 (2회 이상 사용된 경로):")
    for w in weights:
        print(f"  - {w['from_name']} → {w['to_name']}: 가중치 {w['weight']}")

    # 3. 전체 경로 시각화
    print("\n🗺️ 전체 경로 구조:")
    all_paths = graph.query("""
    MATCH (r:ROOT {domain: 'youtube.com'})
    MATCH path = (r)-[:HAS_PAGE|NAVIGATES_TO*..5]->(end:PAGE)
    WHERE NOT (end)-[:NAVIGATES_TO]->()
    RETURN [n in nodes(path) |
        CASE
            WHEN n:ROOT THEN n.domain
            ELSE n.textLabels[0]
        END
    ] as path_nodes
    """)

    for i, path in enumerate(all_paths, 1):
        print(f"  경로 {i}: {' → '.join(path['path_nodes'])}")

In [ ]:
# 다시 테스트
print("🔄 수정된 버전으로 재테스트:")
save_desire_path_fixed(graph, add_metadata_to_path(test_case_1.copy()))
save_desire_path_fixed(graph, add_metadata_to_path(test_case_2.copy()))

# 결과 확인
run_overlap_test(graph, [])  # 이미 저장했으니 분석만

🔄 수정된 버전으로 재테스트:
✅ PAGE: 검색 (youtube.com)
✅ PAGE: 필터 (youtube.com)
✅ PAGE: 오늘 (youtube.com)
✅ PAGE: 첫 번째 동영상 (youtube.com)

✅ 경로 저장 완료!
✅ PAGE: 검색 (youtube.com)
✅ PAGE: 세 번째 동영상 (youtube.com)
✅ PAGE: 좋아요 (youtube.com)

✅ 경로 저장 완료!
🧪 경로 겹침 테스트 시작


📊 테스트 결과 분석:

🔄 공유되는 PAGE 노드:

⚖️ 증가된 가중치 (2회 이상 사용된 경로):
  - youtube.com → 검색: 가중치 2

🗺️ 전체 경로 구조:
  경로 1: youtube.com → 검색 → 필터 → 오늘 → 첫 번째 동영상
  경로 2: youtube.com → 검색 → 세 번째 동영상 → 좋아요
